In [ ]:
%matplotlib inline
import warnings
import inspect
import matplotlib.pyplot as plt
import IPython.display
from cued_sf2_lab.familiarisation import load_mat_img, plot_image
import numpy as np
from typing import Tuple

<figure id="figure-5">
<div style="background-color: white">

![](figures/dwt.svg)
</div>

<figcaption style="text-align: center">

Figure 5: An $L$ level binary discrete wavelet transform.</figcaption></figure>

# 9 The Discrete Wavelet Transform (DWT)

<div class="alert alert-warning alert-block">
    
This notebook is incomplete!</div>

The final method of energy compaction that we shall investigate, is the
discrete wavelet transform. In some ways this attempts to combine the best features of
the Laplacian pyramid and the DCT:

* Like the pyramid, the DWT analyses the image at a range of different
  scales (levels) and employs symmetrical filters;

* Like the DCT, the DWT avoids any expansion in the number of coefficients.

Wavelet theory was evolved by mathematicians during the 1980's. As with the LBT, we shall not attempt to teach this theory here, just illustrate a relatively simple form of it.

Wavelets are short waveforms which are usually the impulse responses of
filters.  Wavelet transforms employ banks of bandpass filters, whose impulse
responses are scaled versions of each other, in
order to get pass-bands in different parts of the frequency spectrum.  If the
impulse response of a filter is scaled in time by a factor $a$, then the
filter frequency response is scaled by the factor $1/a$.  Typically $a = 2$
from one filter to the next, and each bandpass filter is designed to pass a
2:1 range of frequencies (one octave). We can split an image up using wavelets by a process known as a _binary wavelet tree_.

## 9.1 The binary wavelet tree


We start in 1-D with the
simplest possible pair of filters, operating on just two input samples, $x_n$
and $x_{n-1}$.  The two filter outputs, $u_n$ and $v_n$ at time $n$ are
given by:
$$
 u_n = \tfrac{1}{2} (x_n + x_{n-1}) \quad \text{and} \quad
 v_n = \tfrac{1}{2} (x_n - x_{n-1})
$$

The first filter averages adjacent samples, and so rejects the higher
frequency components of $x$, while the second filter differences these
samples, and so rejects the lower frequency components.  These filters are
known as the _analysis_ filter pair, $H_1(z) = \tfrac{1}{2} (1 + z^{-1})$
and $H_2(z) = \tfrac{1}{2} (1 - z^{-1})$.  It is clear that we can recover the
two input samples from the filter outputs using:
$$
 x_n = u_n + v_n \quad \text{and} \quad x_{n-1} = u_n - v_n
$$

Next it is important to note that we need only retain the samples of $u_n$
and $v_n$ at even values of $n$ in order to be able to recover all the
original samples of $x$.  Hence $u$ and $v$ may be decimated 2:1 and still
allow perfect reconstruction of $x$.  If $x$ is a finite length vector (e.g. a
row of image pixels), then $u$ and $v$ are each half as long as $x$, so the
total number of samples is preserved by the transformation.

A wavelet binary tree may be constructed using these filters, by using an
identical pair, $H_1$ and $H_2$, to filter the decimated lowpass signal
$u_{2n}$, to give a pair of outputs, $uu_{2n}$ and $uv_{2n}$, representing
the lower and upper halves of the first low band.  These may again be
decimated 2:1 and still permit perfect reconstruction of $u$.  This process
may be continued as often as desired: each time splitting the lowest band in
two, and decimating the sample rate of the filter outputs by 2:1.  At each
stage the bandwidth of the two lowest filters is halved, and their impulse
responses are doubled in length.  The total number of output samples remains
constant, however many stages are used.

For example, if $f_s$ is the input sample rate, a 3-stage binary tree will
split the input signal bandwidth of 0 to $f_s/2$ into the following four
bands:
$$
0 \rightarrow f_s/16; \ \ f_s/16 \rightarrow f_s/8; \ \ f_s/8 \rightarrow
f_s/4;  \ \ f_s/4 \rightarrow f_s/2.
$$

The very simple filters, given above, do not generate a filter tree with
good characteristics, since the wavelets turn out to be just a pair of
square pulses.  These generate _blocking_ artefacts when used for image
compression (in fact they are equivalent to the 2 point ($N=2$)
DCT). A better set of filters are the LeGall 5 and 3 tap pair,
given by:
$$
 u_n = \tfrac{1}{8} (-x_{n+2} + 2 x_{n+1} + 6 x_n + 2 x_{n-1} - x_{n-2})
  \quad \text{ and }  \quad
 v_{n+1} = \tfrac{1}{4} (-x_{n+2} + 2 x_{n+1} - x_n)
$$

If $u$ and $v$ are decimated by 2 by choosing even $n$ only, the lowband outputs
$u_n$ are centred on the even samples, and the highband outputs $v_{n+1}$ are
centred on the odd samples.  This is very important to allow perfect
reconstruction of $x$ from $u$ and $v$.

The equations for reconstruction may be obtained by solving the above to get:

\begin{align}
 x_n &= \tfrac{1}{2} (-v_{n+1} + 2 u_n - v_{n-1}) \quad \text{and}  \\
 x_{n+1} &= \tfrac{1}{2} (x_{n+2} + 4 v_{n+1} + x_n) =
 \tfrac{1}{4} (-v_{n+3} + 2 u_{n+2} + 6 v_{n+1} + 2 u_n - v_{n-1})
\end{align}

In general, most analysis filters will not yield such simple reconstruction
solutions, and the design of suitable filters is a non-trivial topic that we
shall not cover here.


# 9.2 Applying the DWT to images


As with the DCT, the 2-D DWT may be obtained by applying a 1-D transform to
first the rows and then the columns of an image.

Start by loading the Lighthouse image and defining the two LeGall
filters given above:

In [ ]:
X, _ = load_mat_img(img='lighthouse.mat', img_info='X', cmap_info={'map', 'map2'})
X = X - 128.0
h1 = np.array([-1, 2, 6, 2, -1])/8
h2 = np.array([-1, 2, -1])/4

We can use the function `rowdec` from the pyramid work, to
produce a decimated and lowpass filtered version of the rows of
`X` (remembering to subtract 128 as before) using:

In [ ]:
from cued_sf2_lab.laplacian_pyramid import rowdec
U = rowdec(X, h1)

To get the high-pass image `V`, it is important to align the decimated
samples with the odd columns of `X` (assuming the first column is $n = 0$)
whereas `U` is aligned with the even columns.  To do this we use a
slightly modified version of `rowdec`, called `rowdec2`.

In [ ]:
from cued_sf2_lab.laplacian_pyramid import rowdec2
V = rowdec2(X, h2)

<div class="alert alert-block alert-danger">

Display `U` and `V` to see the outputs of the first filter pair
and comment on their relative energies (or standard deviations). Note that `U` and `V` are half the width of `X`, but that `U` is otherwise similar to `X`.</div>

In [ ]:
fig, ax = plt.subplots(1, 2)

print(f'Energy in U: {np.sum(U**2)}, in V: {np.sum(V**2)} ')

plot_image(U, ax=ax[0])
plot_image(V, ax=ax[1])

Now filter the columns of `U` and `V` using `rowdec
/ rowdec2` with the transpose operator:

In [ ]:
UU = rowdec(U.T, h1).T
UV = rowdec2(U.T, h2).T
VU = rowdec(V.T, h1).T
VV = rowdec2(V.T, h2).T

<div class="alert alert-block alert-danger">
    
Display `np.block([[UU, VU], [UV, VV]])`, and comment
on what sort of edges or features are selected by each filter. You may need to multiply the high-pass images by a factor $k > 1$ to display them clearly. Why is this?</div>

In [ ]:
k = 2 # Scale factor to make highpass images visible
# probably because of how little energy they contain

R = np.block([[UU, k*VU], [k*UV, k*VV]])

plot_image(R)

We must now check that it is possible to recover the image from
these sub-images, using reconstruction filters, `g1` and `g2`, and the functions, `rowint` and `rowint` (which
is modified in a similar way to `rowdec2` to allow correct
alignment of the high-pass samples). To reconstruct `Ur` and
`Vr` from `UU`, `UV`, `VU` and `VV` use:

In [ ]:
from cued_sf2_lab.laplacian_pyramid import rowint, rowint2

g1 = np.array([1, 2, 1])/2
g2 = np.array([-1, -2, 6, -2, -1])/4
Ur = rowint(UU.T, g1).T + rowint2(UV.T, g2).T
Vr = rowint(VU.T, g1).T + rowint2(VV.T, g2).T

Note the gain of 2 in the reconstruction filters, `g1` and
`g2` (to compensate for losing half the samples in the
decimation / interpolation processes).   These filters are also
not quite the same as those that might be inferred from the
equations for $x_n$ and $x_{n+1}$ on the previous page.  This is
because `g1` defines how {\it only} the $u$ samples contribute
both to the even and odd samples of $x$, while `g2` defines
how the $v$ samples contribute.

Check that `Ur` and `Vr` are the same as `U` and
`V`, and then reconstruct `Xr` from these:

In [ ]:
print(np.testing.assert_equal(Ur, U))
print(np.testing.assert_equal(Vr, V))

In [ ]:
Xr = rowint(Ur,g1) + rowint2(Vr,g2)

Check that `Xr` is the same as `X`.

In [ ]:
print(np.testing.assert_equal(X, Xr))

The above operations are a bit tedious to repeat if we want to
apply the DWT recursively to obtain several levels of filtering,
so we have written a pair of functions, `dwt` and `idwt`, to perform the 2-D analysis and reconstruction
operations. Examine these to see that they perform the same
operations as above, except that the transformed sub-images are
stored as parts of a single matrix, the same size as `X`,
rather than as separate matrices.

In [ ]:
from cued_sf2_lab.dwt import dwt
IPython.display.Code(inspect.getsource(dwt), language="python")

In [ ]:
from cued_sf2_lab.dwt import idwt
IPython.display.Code(inspect.getsource(idwt), language="python")

You can check their operation as below::

In [ ]:
Y = dwt(X)
Xr = idwt(Y)

fig, axs = plt.subplots(1, 2)
plot_image(Y, ax=axs[0])
axs[0].set(title="Y")
plot_image(Xr, ax=axs[1])
axs[1].set(title="Xr");

`Y` should be the same as the composite `[UU VU; UV VV]` image that
you displayed earlier, and `Xr` should be the same as `X`.

Now implement a multilevel DWT by first applying `dwt` to
`X` using:

```python
m=256
Y=dwt(X)
plot_image(Y, ax=some_axis)
```

and then iteratively apply `dwt` to the top left sub-image
of `Y` by repeating:
```python
m = m//2
Y[:m,:m] = dwt(Y[:m,:m])
plot_image(Y, ax=some_axis)
```

In [ ]:
fig, ax = plt.subplots(1, 5, figsize=(15, 6))

Y = X.copy()

L = 5

for i in range(L):
    m = 256//(2**i)
    Y[:m,:m] = dwt(Y[:m,:m])
    plot_image(Y, ax=ax[i])

We now have the image split using a binary wavelet tree (stricly a
quaternary tree in 2-D).  Write
similar iterative code to that given above, which can reconstruct
the image from the final set of `Y` sub-images after a 4-level
wavelet transform. Check that your reconstructed image is the
same as `X`.

In [ ]:
Yr = Y.copy()

fig, ax = plt.subplots(1, 5, figsize=(15, 6))

for i in range(L):
    m = 256//(2**(L - i - 1))
    Yr[:m,:m] = idwt(Yr[:m,:m])
    plot_image(Yr, ax=ax[i])


## 9.3 Quantisation and coding efficiency

First rewrite the sequences of operations required to perform
$n$ levels of DWT and inverse DWT as two separate M-files, `nlevdwt` and `nlevidwt`. `nlevdwt` should transform
`X` into `Y`, and `nlevidwt` should inverse
transform a quantised set of sub-images `Yq` into the
reconstructed image `Z`.  Check your functions by ensuring
that `Z` is the same as `X` if `Yq = Y`.

In [ ]:
def nlevdwt(X, n):
    
    Y = X.copy()

    for i in range(n):

        m = 256//(2**(i))
        Y[:m,:m] = dwt(Y[:m,:m])

    return Y

def nlevidwt(Y, n):

    Z = Y.copy()

    for i in range(n):

        m = 256//(2**(n - i - 1))
        Z[:m,:m] = idwt(Z[:m,:m])
    
    return Z

In [ ]:
# Check zero reconstruction error

N = 4

Z = nlevidwt(nlevdwt(X, N), N)

print(np.max(np.abs(X - Z)))

Now design a function, `quantdwt`, which will quantise the
sub-images of `Y` to give `Yq` and calculate their
entropy.  The sub-images at each level `i` of the DWT should
be quantised according to a $3 \times (n+1)$ matrix `dwtstep[k,i]` of
step-sizes, where $\mathtt{k}=\left\{0,1,2\right\}$ corresponds to each of the three high-pass images at level `i` (top right, bottom left, and bottom right, respectively), and the final low-pass image is quantised with `dwtstep[0,n]`. This matrix will be populated either with the same number in all elements (for equal-step-size quantisation) or a range of different numbers (for equal-MSE quantisation). The entropies for each sub-image should be stored in a similar $3 \times (n+1)$ matrix `dwtent[k,i]`.

In [ ]:
# Image size func

def img_size(img):
    return bpp(img) * img.shape[0] * img.shape[1]

In [ ]:
from cued_sf2_lab.laplacian_pyramid import quantise, bpp

# Quantise DWT image

def quantdwt(Y: np.ndarray, dwtstep: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """
    Parameters:
        Y: the output of `dwt(X, n)`
        dwtstep: an array of shape `(3, n+1)`
    Returns:
        Yq: the quantized version of `Y`
        dwtent: an array of shape `(3, n+1)` containing the entropies
    """
    
    n = dwtstep.shape[1] - 1 # number of times binary DWT is applied
    Yq = np.zeros_like(Y)
    dwtent = 0

    for i in range(n):

        m = 256//(2**i) # 256, 128, 64 ... 
        h = m//2 # Midpoint: 128, 64, 32 ...

        # Quantising
        Yq[:h, h:m] = quantise(Y[:h, h:m], dwtstep[0][i]) # Top right 
        Yq[h:m, :h] = quantise(Y[h:m, :h], dwtstep[1][i]) # Bottom left
        Yq[h:m, h:m] = quantise(Y[h:m, h:m], dwtstep[2][i]) # Bottom right

        # Find entropy of each from above
        dwtent += img_size(Yq[:h, h:m]) + img_size(Yq[h:m, :h]) + img_size(Yq[h:m, h:m])


    # Final low pass image
    m = 256//(2**n)
    Yq[:m, :m] = quantise(Y[:m, :m], dwtstep[0, n])
    dwtent += img_size(Yq[:m, :m])
    
    return Yq, dwtent

Using these functions, for a given number of levels $n$ (typically
between 3 and 5), you should generate `Y`, quantise it to
give `Yq` and reconstruct `Z` from `Yq`.

In [ ]:
N = 4

dwtstep = np.ones((3, N+1)) * 17

Y = nlevdwt(X, N)
Yq, ent = quantdwt(Y, dwtstep)
Z = nlevidwt(Yq, N)

print(f'Compression Ratio = {img_size(quantise(X, 17))/ent}')

fig, ax = plt.subplots(1, 2, figsize=(16, 5))
plot_image(X, ax=ax[0])
plot_image(Z, ax=ax[1])

# This produces high compression high error with 17 as base step

All of our experiments thus far have been performed on only one image. At this stage it is worth starting to experiment with the additional `Bridge` image, as well as Lighthouse. Bridge contains a lot more fine detail and may not lead to the same conclusions regarding performance.

In [ ]:
Xb, _ = load_mat_img(img='bridge.mat', img_info='X', cmap_info={'map'})
Xb = Xb - 128.0

In [ ]:
fig, ax = plt.subplots()
plot_image(Xb, ax=ax)
ax.set(title="bridge.mat");


<div class="alert alert-block alert-danger">

Investigate the performance of
both an equal-step-size and an equal-MSE scheme (follow a similar procedure as you used for the Laplacian Pyramid to find the appropriate step-size ratios). Hence determine how many levels of DWT are reasonably optimal for the Lighthouse and Bridge images. Also evaluate the subjective quality of your reconstructed images, and comment on how this depends on $n$ and on the way that step-sizes are assigned
to the different levels. Once again, for each image choose quantisation steps such that you match the rms error to that for direct quantisation with a step-size of 17.</div>

In [ ]:
# energies between layers

n = 7

B = np.zeros_like(X)
E = []

Y = nlevdwt(B, n)

for n in range(n):

    Yp = Y.copy()

    m = 256//(2**(i+2))
    Yp[m, m] = 100
    Z = nlevidwt(Yp, n)

    E.append(np.sum(Z**2))


for i in range(len(E)-1):

    print(f'Energy ratio, layer {i+1} / layer {i}: \t{E[i+1]/E[i]:.4}')

# Tends towards 4 again? correct approach?

In [ ]:
from scipy import optimize

# Modified laplacian pyramid optimisation function for DWT

BASE_STEP = 17

def optimum_step_size(n, ratio=1):

    Xq = quantise(X, BASE_STEP)
    directq_error = np.std(X - Xq)

    def error_difference(steps):

        Y = nlevdwt(X, n) # Decompose

        # Set quantisation levels according to ratio and initial step
        dwtstep = np.array([np.ones((1, 3))[0]*steps*(ratio**i) for i in range(n + 1)]).T
        
        Yq, ent = quantdwt(Y, dwtstep) # Quantise
        Z = nlevidwt(Yq, n) # Decode

        # Return difference in errors
        zq_error = np.std(X - Z)
        return (directq_error - zq_error) ** 2

    res = optimize.minimize_scalar(error_difference, bounds=(1, 256) )
    return res.x

optimum_step_size(4, ratio=1/2)

In [ ]:
# Varying DWT layer depth maintaining reference error with const. and equal MSE schemes

Xq = quantise(X, BASE_STEP)
entr = img_size(Xq)

def optimise_depth(X):

    fig, axs = plt.subplots(2, 6, sharex=True, sharey=True, figsize=(15, 6))

    for i, n in enumerate(range(1, 7)):

        Y = nlevdwt(X, n)

        for j, r in enumerate([1, 1/2]):

            steps = optimum_step_size(n, ratio=r)

            dwtstep = np.array([np.ones((1, 3))[0]*steps*(r**i) for i in range(n + 1)]).T

            Yq, ent = quantdwt(Y, dwtstep)

            Z = nlevidwt(Yq, n)

            print(f'{"Constant" if j == 0 else "Eq. MSE"} {n} layers\t cr = {entr/ent:.4}\t init. step = {steps:.4}\t error = {np.std(Z - X):.4}')

            plot_image(Z, ax=axs[j][i])
    

optimise_depth(X)


In [ ]:
# Same for bridge image

optimise_depth(Xb)

## 9.4 Second Interim Report

This report should include the new results from the DCT, LBT and DWT energy
compaction methods in a format that will allow them to be compared with each other and contrasted to the
Laplacian pyramid work in your first report.  Again try to answer questions
raised in the text, and also include discussion of any topics that have led to
unexpected results or have proved particularly interesting.